In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
import os
os.chdir('/gdrive/MyDrive/Colab Notebooks/')
os.chdir('/gdrive/Othercomputers/iMac/SSAC/12메인프로젝트/Project_Team2/속초고성_전기차충전소_입지선정/')

Mounted at /gdrive


In [22]:
!pip install geopandas
!pip install pydeck
!pip install cufflinks
!pip install mip
!pip install geojson

In [8]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd # 설치가 조금 힘듭니다. 어려우시면 https://blog.naver.com/PostView.nhn?blogId=kokoyou7620&logNo=222175705733 참고하시기 바랍니다.
import folium
import shapely 
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sklearn.cluster
import tensorflow as tf  # 설치 따로 필요합니다. https://chancoding.tistory.com/5 참고 하시면 편해요.

#from geoband import API         이건 설치 필요 없습니다.

import pydeck as pdk                  # 설치 따로 필요합니다.
import os

import pandas as pd


import cufflinks as cf                 # 설치 따로 필요합니다.   
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'

import numpy as np
from shapely.geometry import Polygon, Point
from numpy import random

#최적화 solver
import time
from mip import Model, xsum, maximize, BINARY  # 설치 따로 필요합니다.

In [9]:
#Pydeck 사용을 위한 함수 정의
import geopandas as gpd 
import shapely # Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환합니다. 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 




---------------------------------
# I. EDA

## 1. 유동인구분포


**목적**

**분석방법**
- 격자별 상권 밀집도: 격자에 포함된 상가 수 / 동단위 총 상가 수
- 격자별 유동인구: 읍면동 유동인구 * 격자별 상권 밀집도 

**분석 데이터 종류**
- 읍면동 유동인구 현황
- 속초고성 상권정보
- 속초고성 100X100격자지도

**분석 설명**
- 인구수에 따라 색이 다르게 시각화

---------------------------------
## 2. 상권정보 분석

**목적: **

**분석 데이터 종류**


---------------------------------
## 3. 관광정보 분석


---------------------------------
## 4. 교통량 분석


**목적: 주거시간과 업무시간의 교통량 분석**

**분석 데이터 종류**
- df_10: 10.강원도속초시,고성군_상세도로망.geojson
- df_11: 11.평일_일별_시간대별_추정교통량(대체데이터).csv"

**분석 방법**
- 전기자동차는 승용차만을 고려한다고 가정
- 교통량: 11시, 14시의 승용차 교통량


**분석그래프 설명**
- 붉은 색일수록 혼잡강도가 높은 것이다.
- 선이 굵을 수록 혼잡강도가 높은 것이며, 얇을 수록 낮은 것이다

In [10]:
df_10 = gpd.read_file('10.강원도속초시,고성군_상세도로망.json')
df_11 = pd.read_csv('11.평일_일별_시간대별__추정교통량.csv')

In [11]:
df = df_10
df = df_10
df['coordinate'] = df['geometry'].buffer(0.001).apply(polygon_to_coordinates) 
df = pd.DataFrame(df) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔줘야 합니다. 
df['정규화도로폭'] = df['width'].apply(int) / df['width'].apply(int).max()

In [12]:
# 여행객들이 11시에 관광지로 움직일 것으로 가정
# 승용차만 고려
df_11_time11 = df_11[df_11['시간적범위']==11]

df_10_ = []
for i in df_10.link_id:
    df_10_.append([i,sum(df_11_time11[df_11_time11['link_id'].apply(str).str.contains(i)]['승용차'])])
    
df_10_ = pd.DataFrame(df_10_).fillna(0)
df_10_.columns = ["link_id", "교통량"]
df_10_11_time11=pd.merge(df, df_10_,on = 'link_id' )

# 교통량 합이 가장 높은 도로
df_10_11_time11.iloc[df_10_11_time11["교통량"].sort_values(ascending=False).index].reindex().head()

,link_id,up_f_node,up_t_node,dw_f_node,dw_t_node,max_speed,road_name,road_no,road_rank,link_type,pavement,road_type,facil_name,tg_name,up_lanes,dw_lanes,lanes,oneway,length,width,Level,spot_id,hov_lane,shov_lane,car_lane,num_cross,barrier,up_its_id,dw_its_id,sido_id,sigungu_id,emd_id,up_vdf,dw_vdf,up_i_speed,dw_i_speed,up_capa,dw_capa,emme_check,up_c_year,dw_c_year,end_year,m_check,m_date,up_v_Link,dw_v_Link,geometry,coordinate,정규화도로폭,교통량
1171,685708012,498029,498226,498226,498029,60,동해대로,7,103,32768,0,0,,,3,3,6,0,0.286,4,5,,0,0,0,0,2,2550000900,2550001000,32000,32060,3206059,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570801201,68570801202,"LINESTRING (128.60175 38.18087, 128.60110 38.1...","[[128.6001477024162, 38.18265263329313], [128....",1.00,1223.51
1429,685711642,498088,498390,498390,498088,70,미시령로,56,105,32768,0,0,,,2,2,4,0,0.28,3,5,,0,0,0,0,5,2550011900,2550012000,32000,32060,3206057,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68571164201,68571164202,"LINESTRING (128.53039 38.20262, 128.53029 38.2...","[[128.53026003516123, 38.20162487506265], [128...",0.75,1208.52
964,685706710,498088,498089,498089,498088,70,미시령로,56,105,32768,0,3,콩꽃마을교,,2,2,4,0,0.146,3,5,,0,0,0,0,5,2550012200,2550012100,32000,32060,3206057,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570671001,68570671002,"LINESTRING (128.53039 38.20262, 128.53145 38.2...","[[128.53140296639802, 38.20364111955838], [128...",0.75,1206.52
969,685706725,498089,498090,498090,498089,70,미시령로,56,105,32768,0,0,,,2,2,4,0,0.094,3,5,,0,0,0,0,5,2550011400,2550011300,32000,32060,3206057,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570672501,68570672502,"LINESTRING (128.53205 38.20271, 128.53311 38.2...","[[128.53297688342383, 38.20384557204121], [128...",0.75,1202.52
1769,685714555,498395,622728,622728,498395,70,미시령로,56,105,32768,0,0,,,2,2,4,0,0.575,3,5,,0,0,0,0,5,2550012303,2550012402,32000,32060,3206057,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68571455501,68571455502,"LINESTRING (128.52713 38.20382, 128.52696 38.2...","[[128.52671899787921, 38.20289803284105], [128...",0.75,1195.52


In [13]:
layer = pdk.Layer( 'PathLayer', 
                  df_10_11_time11, 
                  get_path='coordinate', 
                  get_width='교통량/2', 
                  get_color='[255, 255 * 정규화도로폭, 120]', 
                  pickable=True, auto_highlight=True 
                 ) 



center = [128.5918, 38.27701]
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 
r = pdk.Deck(layers=[layer], initial_view_state=view_state
            ) 
r.to_html('교통량_11.html')


Output hidden; open in https://colab.research.google.com to view.

### 분석 결과

- 출근시간을 피한 오전 11시에 교통량이 많은 곳은 관광객이 많이 가는 곳으로 간주하였다.

In [14]:
# 여행객들이 14시에 관광지로 움직일 것으로 가정
df_11_time14=df_11[df_11['시간적범위']==14]

df_10_ = []
for i in df_10.link_id:
    df_10_.append([i,sum(df_11_time14[df_11_time14['link_id'].apply(str).str.contains(i)]['승용차'])])
    
df_10_ = pd.DataFrame(df_10_).fillna(0)
df_10_.columns = ["link_id", "교통량"]
df_10_11_time14=pd.merge(df, df_10_,on = 'link_id' )

# 교통량 합이 가장 높은 도로
df_10_11_time14.iloc[df_10_11_time14["교통량"].sort_values(ascending=False).index].reindex().head()

,link_id,up_f_node,up_t_node,dw_f_node,dw_t_node,max_speed,road_name,road_no,road_rank,link_type,pavement,road_type,facil_name,tg_name,up_lanes,dw_lanes,lanes,oneway,length,width,Level,spot_id,hov_lane,shov_lane,car_lane,num_cross,barrier,up_its_id,dw_its_id,sido_id,sigungu_id,emd_id,up_vdf,dw_vdf,up_i_speed,dw_i_speed,up_capa,dw_capa,emme_check,up_c_year,dw_c_year,end_year,m_check,m_date,up_v_Link,dw_v_Link,geometry,coordinate,정규화도로폭,교통량
1171,685708012,498029,498226,498226,498029,60,동해대로,7,103,32768,0,0,,,3,3,6,0,0.286,4,5,,0,0,0,0,2,2550000900,2550001000,32000,32060,3206059,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570801201,68570801202,"LINESTRING (128.60175 38.18087, 128.60110 38.1...","[[128.6001477024162, 38.18265263329313], [128....",1.00,1248.50
1429,685711642,498088,498390,498390,498088,70,미시령로,56,105,32768,0,0,,,2,2,4,0,0.28,3,5,,0,0,0,0,5,2550011900,2550012000,32000,32060,3206057,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68571164201,68571164202,"LINESTRING (128.53039 38.20262, 128.53029 38.2...","[[128.53026003516123, 38.20162487506265], [128...",0.75,1171.54
964,685706710,498088,498089,498089,498088,70,미시령로,56,105,32768,0,3,콩꽃마을교,,2,2,4,0,0.146,3,5,,0,0,0,0,5,2550012200,2550012100,32000,32060,3206057,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570671001,68570671002,"LINESTRING (128.53039 38.20262, 128.53145 38.2...","[[128.53140296639802, 38.20364111955838], [128...",0.75,1169.53
969,685706725,498089,498090,498090,498089,70,미시령로,56,105,32768,0,0,,,2,2,4,0,0.094,3,5,,0,0,0,0,5,2550011400,2550011300,32000,32060,3206057,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570672501,68570672502,"LINESTRING (128.53205 38.20271, 128.53311 38.2...","[[128.53297688342383, 38.20384557204121], [128...",0.75,1166.53
1015,685706892,498126,498127,498127,498126,70,미시령로,56,105,32768,0,3,원암대교,,2,2,4,0,0.107,3,5,,0,0,0,0,5,2550011600,2550011500,32000,32400,3240033,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570689201,68570689202,"LINESTRING (128.50924 38.20582, 128.51041 38.2...","[[128.51063834666533, 38.20651444681091], [128...",0.75,1158.43


In [15]:
layer = pdk.Layer( 'PathLayer', 
                  df_10_11_time14, 
                  get_path='coordinate', 
                  get_width='교통량/2', 
                  get_color='[255, 255 * 정규화도로폭, 120]', 
                  pickable=True, auto_highlight=True 
                 ) 
center = [128.5918, 38.27701] # 센터 [128.5918, 38.27701]
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 
r = pdk.Deck(layers=[layer], initial_view_state=view_state
            ) 
r.to_html('교통량_14.html')

Output hidden; open in https://colab.research.google.com to view.

### 분석 결과
- 출퇴근 시간을 피한 오후 14시에 교통량이 많은 곳은 관광객이 많이 가는 지역일 것으로 기대
- 급속 충전소는 관광지역 설치가 효과적이라 판단

-------------------------------------

---------------------------------
## 5. 혼잡빈도강도, 혼잡시간강도 분석


**목적: 혼잡빈도강도와 혼잡시간빈도를 분석하여 차량이 많은 위치 파악**

**분석 데이터 종류**
- df_10: 10.강원도속초시,고성군_상세도로망.json
- df_12: 12.평일_혼잡빈도강도_강원도 속초시, 고성군.csv"
- df_13: 13.평일_혼잡시간강도_강원도 속초시, 고성군.csv"

**분석 방법**
- 혼잡빈도강도합: 양방향의 혼잡빈도강도 합
- 정규화도로폭: width변수를 이용하여 정규화


**분석그래프 설명**
- 도로폭이 넓을 수록 노란색이며 좁을 수록 붉은색이다.
- 선이 굵을 수록 혼잡빈도강도가 높은 것이며, 얇을 수록 낮은 것이다

In [16]:
df_10= gpd.read_file("10.강원도속초시,고성군_상세도로망.json")
df_12= pd.read_csv("12.평일_혼잡빈도강도_강원도 속초시, 고성군.csv")
df_13= pd.read_csv("13.평일_혼잡시간강도_강원도 속초시, 고성군.csv")

In [17]:
# 혼합빈도강도 양방향 총 합
df_10_ = []
for i in df_10.link_id:
    df_10_.append([i,sum(df_12[df_12['link_id'].apply(str).str.contains(i)].혼잡빈도강도)])
    
df_10_ = pd.DataFrame(df_10_).fillna(0)
df_10_.columns = ["link_id", "혼잡빈도강도합"]
df_10_12=pd.merge(df, df_10_,on = 'link_id' )

# 혼잡빈도강도 합이 가장 높은 도로
df_10_12.iloc[df_10_12["혼잡빈도강도합"].sort_values(ascending=False).index].reindex().head()

,link_id,up_f_node,up_t_node,dw_f_node,dw_t_node,max_speed,road_name,road_no,road_rank,link_type,pavement,road_type,facil_name,tg_name,up_lanes,dw_lanes,lanes,oneway,length,width,Level,spot_id,hov_lane,shov_lane,car_lane,num_cross,barrier,up_its_id,dw_its_id,sido_id,sigungu_id,emd_id,up_vdf,dw_vdf,up_i_speed,dw_i_speed,up_capa,dw_capa,emme_check,up_c_year,dw_c_year,end_year,m_check,m_date,up_v_Link,dw_v_Link,geometry,coordinate,정규화도로폭,혼잡빈도강도합
2439,694100981,503157,503263,503263,503157,0,간성로,0,107,32768,0,0,,,1,1,2,0,0.065,2,6,,0,0,0,0,0,0,0,32000,32400,3240011,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,69410098101,69410098102,"LINESTRING (128.46937 38.37929, 128.46929 38.3...","[[128.47028065186726, 38.37857279273677], [128...",0.50,200
2641,694103002,503170,503201,503201,503170,0,간성북로7번길,0,107,32768,0,0,,,1,1,2,0,0.124,2,6,,0,0,0,0,0,0,0,32000,32400,3240011,0,0,0.0,0.0,0,0,0,0,0,0,0,20171231,69410300201,69410300202,"LINESTRING (128.47035 38.38032, 128.47040 38.3...","[[128.47132000147764, 38.3805642182566], [128....",0.50,200
839,685706142,497956,497973,497973,497956,60,동해대로,7,103,32768,0,0,,,3,3,6,0,0.055,4,5,,0,0,0,0,15,2550004200,2550004100,32000,32060,3206058,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570614201,68570614202,"LINESTRING (128.59633 38.18998, 128.59696 38.1...","[[128.59688406320052, 38.19102341198306], [128...",1.00,200
838,685706140,497500,497501,497501,497500,30,동해대로,7,103,4,0,0,,,3,3,6,0,0.066,4,5,,0,0,0,0,15,2550004500,2550004600,32000,32060,3206058,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68570614001,68570614002,"LINESTRING (128.59939 38.19003, 128.59929 38.1...","[[128.5989277442983, 38.18913625215332], [128....",1.00,200
106,684700724,496615,496623,496623,496615,0,,0,107,34816,0,0,,,1,1,1,0,0.015,1,6,,0,0,0,0,0,0,0,32000,32400,3240033,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68470072401,68470072402,"LINESTRING (128.49435 38.21089, 128.49434 38.2...","[[128.49339124524636, 38.21059290520445], [128...",0.25,200


In [18]:
layer = pdk.Layer( 'PathLayer', 
                  df_10_12, 
                  get_path='coordinate', 
                  get_width='혼잡빈도강도합', 
                  get_color='[255, 255 * 정규화도로폭, 120]', 
                  pickable=True, auto_highlight=True 
                 ) 
center = [128.5918, 38.27701] # 센터 [128.5918, 38.27701]
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
#             mapbox_key = 'sk.eyJ1IjoieW9vYnl1bmdjaHVsIiwiYSI6ImNrd245YnMwZzFiMnEycHBkc2gzbzkzd3AifQ.sc9Gmo56AsAHzJ2B3wCkXg') 
r.to_html('혼잡빈도강도.html')

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# 혼합시간강도 양방향 총 합
df_10_ = []
for i in df_10.link_id:
    df_10_.append([i,sum(df_13[df_13['link_id'].apply(str).str.contains(i)].혼잡시간강도)])
    
df_10_ = pd.DataFrame(df_10_).fillna(0)
df_10_.columns = ["link_id", "혼잡시간강도합"]
df_10_13=pd.merge(df, df_10_,on = 'link_id' )
# 혼잡시간강도 합이 가장 높은 도로
df_10_13.iloc[df_10_13["혼잡시간강도합"].sort_values(ascending=False).index].reindex().head()

,link_id,up_f_node,up_t_node,dw_f_node,dw_t_node,max_speed,road_name,road_no,road_rank,link_type,pavement,road_type,facil_name,tg_name,up_lanes,dw_lanes,lanes,oneway,length,width,Level,spot_id,hov_lane,shov_lane,car_lane,num_cross,barrier,up_its_id,dw_its_id,sido_id,sigungu_id,emd_id,up_vdf,dw_vdf,up_i_speed,dw_i_speed,up_capa,dw_capa,emme_check,up_c_year,dw_c_year,end_year,m_check,m_date,up_v_Link,dw_v_Link,geometry,coordinate,정규화도로폭,혼잡시간강도합
3280,695100353,503735,503771,503771,503735,40,동해대로,7,103,4,0,0,,,2,2,4,0,0.021,3,5,,0,0,0,0,2,2660008000,2660007900,32000,32400,3240032,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,69510035301,69510035302,"LINESTRING (128.50237 38.37233, 128.50236 38.3...","[[128.5033601550288, 38.37208319161159], [128....",0.75,200
2641,694103002,503170,503201,503201,503170,0,간성북로7번길,0,107,32768,0,0,,,1,1,2,0,0.124,2,6,,0,0,0,0,0,0,0,32000,32400,3240011,0,0,0.0,0.0,0,0,0,0,0,0,0,20171231,69410300201,69410300202,"LINESTRING (128.47035 38.38032, 128.47040 38.3...","[[128.47132000147764, 38.3805642182566], [128....",0.50,200
2204,685804048,498521,498868,498868,498521,80,동해대로,7,103,4,0,0,,,2,2,4,0,0.016,3,5,,0,0,0,0,5,2660006500,2660006600,32000,32400,3240032,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,68580404801,68580404802,"LINESTRING (128.52419 38.32990, 128.52417 38.3...","[[128.52317993633602, 38.329887409760836], [12...",0.75,200
3285,695100386,503773,503782,503782,503773,0,,0,107,32768,0,0,,,1,1,2,0,0.03,2,6,,0,0,0,0,0,0,0,32000,32400,3240032,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,69510038601,69510038602,"LINESTRING (128.51033 38.35429, 128.51058 38.3...","[[128.51118273676298, 38.354895310074134], [12...",0.50,200
2645,694103044,503289,503413,503413,503289,0,고성중앙길,0,107,34816,0,0,,,1,1,2,0,0.035,2,6,,0,0,0,0,0,0,0,32000,32400,3240011,1,0,0.0,0.0,0,0,0,0,0,0,0,20171231,69410304401,69410304402,"LINESTRING (128.46801 38.38016, 128.46775 38.3...","[[128.46844422281328, 38.37919333706284], [128...",0.50,200


In [20]:
layer = pdk.Layer( 'PathLayer', 
                  df_10_13, 
                  get_path='coordinate', 
                  get_width='혼잡시간강도합', 
                  get_color='[255, 255 * 정규화도로폭, 120]', 
                  pickable=True, auto_highlight=True 
                 ) 
center = [128.5918, 38.27701] # 센터 [128.5918, 38.27701]
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
#             mapbox_key = "pk.eyJ1IjoiamNsYXJhODExIiwiYSI6ImNrZzF4bWNhdTBpNnEydG54dGpxNDEwajAifQ.XWxOKQ-2HqFBVBYa-XoS-g") 
r.to_html()

Output hidden; open in https://colab.research.google.com to view.

---------------------------------
## 6. 급속충전소 설치가능 장소 필터링

**목적: 급속 충전소의 경우 사유지는 제외 해야 하므로 설치 가능 장소 필터링 필요**

**분석 데이터 종류**
- df_14: 14.소유지정보.geojson

**분석 방법**
- 급속충전소 설치 가능 장소 필터링
- 소유구분 코드: 국유지, 시/군 선택
- 지목코드: 임야, 염전, 도로, 철도 용지, 제방, 하천, 유지 제외
- 설치가능장소에 포함되는 기준 point 추출

**분석 설명**
- 사유지를 포함한 임야, 염전, 도로, 철도 용지, 제방, 하천과 같이 설치가 부적절 한 곳을 필터링 한 multipolygone을 시각화하였다.
- 앞서 도출한 인구현황 100X100 Point 데이터셋에서 설치가능한 장소에 해당하는 point를 추출하였다.


In [23]:
df_14= gpd.read_file("14.소유지정보.geojson") # geojson -> json
# 급속 충전소 설치 가능 장소
df_14_=df_14[df_14['소유구분코드'].isin(['02','04'])] #소유구분코드: 국유지, 시/군
df_14_possible=df_14[df_14['소유구분코드'].isin(['02','04']) 
      & (df_14['지목코드'].isin(['05','07','14','15','16','17',
                             '18','19','20','27' ])==False)] # 임야, 염전, 도로, 철도 용지, 제방, 하천 제외 

# geometry to coordinates
df_14_possible['coordinates'] = df_14_possible['geometry'].apply(polygon_to_coordinates) 

# 설치가능한 모든 polygone을 multipolygone으로 묶음
from shapely.ops import cascaded_union
boundary = gpd.GeoSeries(cascaded_union(df_14_possible['geometry'].buffer(0.001)))

from geojson import Feature, FeatureCollection, dump
MULTIPOLYGON = boundary[0]

features = []
features.append(Feature(geometry=MULTIPOLYGON, properties={"col": "privat"}))
feature_collection = FeatureCollection(features)
with open('geo_possible.geojson', 'w') as f:
    dump(feature_collection, f)

geo_possible= gpd.read_file("geo_possible.geojson")

In [24]:
# 브로드캐스팅을 이용한 요소합 (평행이동)
# 요소합 진행 후, 마지막 데이터를 list로 형변환
v = np.array([-0.0022, 0.0027])
for i in range(len(df_14_possible["coordinates"])):
    for j in range(len(df_14_possible["coordinates"].iloc[i])):
                   df_14_possible["coordinates"].iloc[i][j] = list(df_14_possible["coordinates"].iloc[i][j] + v)
df_14_possible["coordinates"]

155       [[128.59391565658433, 38.20214766278363], [128...
171       [[128.585141862336, 38.19994343313207], [128.5...
175       [[128.5837371452338, 38.198010415329684], [128...
209       [[128.59424070341274, 38.20205129243292], [128...
252       [[128.57162130069344, 38.1855029481736], [128....
                                ...                        
131766    [[128.4337296151357, 38.396988963771946], [128...
131771    [[128.54718522324893, 38.292966798373286], [12...
131786    [[128.51166181817612, 38.29924204302354], [128...
131797    [[128.4545407773845, 38.445945922088676], [128...
131808    [[128.44809318724057, 38.382924225788145], [12...
Name: coordinates, Length: 6631, dtype: object

In [25]:
layer = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_14_possible, # 시각화에 쓰일 데이터프레임
                  #df_result_fin[df_result_fin['val']!=0],
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[255, 0, 127]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 

# Set the viewport location 
center = [128.5918, 38.27701] # 센터 [128.5918, 38.27701]
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 


# Render 
r = pdk.Deck(layers=[layer], initial_view_state=view_state,
            ) 

    
r.to_html('충전소설치가능위치.html')

Output hidden; open in https://colab.research.google.com to view.

# II. 입지선정지수 개발

## 1. 지역특성 요소 추출

### 100X100 Point 중 설치 가능 한 Point 필터링 
- 100X100 중 설치 가능한 multipolygone에 있는 point를 필터링하는 시간이 굉장히 오래 소요됨(약 1시간)
- 따라서 이를 모두 계산한 'within_points.csv'를 따로 저장하여 첨부함
- df_result로 최종 분석 할 데이터셋을 만듦

In [26]:
# 격자별 인구 현황
df_08= gpd.read_file("08.속초-고성_격자별인구현황.json")

# val 열 na 제거
df_08['val'] = df_08['val'].fillna(0)

# geometry를 coordinate 형태로 적용
df_08['coordinates'] = df_08['geometry'].apply(polygon_to_coordinates) #pydeck 을 위한 coordinate type

# 100X100 grid에서 central point 찾기
df_08_list = []
df_08_list2 = []
for i in df_08['geometry']:
    cent = [[i.centroid.coords[0][0],i.centroid.coords[0][1]]]
    df_08_list.append(cent)
    df_08_list2.append(Point(cent[0]))
df_08['coord_cent'] = 0
df_08['geo_cent'] = 0
df_08['coord_cent']= pd.DataFrame(df_08_list) # pydeck을 위한 coordinate type
df_08['geo_cent'] = df_08_list2 # geopandas를 위한 geometry type

# 쉬운 분석을 위한 임의의 grid id 부여
df_08['grid_id']=0
idx = []
for i in range(len(df_08)):
    idx.append(str(i).zfill(5))
df_08['grid_id'] = pd.DataFrame(idx)
df_08

,gid,lbl,val,geometry,coordinates,coord_cent,geo_cent,grid_id
0,라아830496,30.00,30.0,"POLYGON ((128.45110 38.44316, 128.45225 38.443...","[[128.4511011187016, 38.4431625579364], [128.4...","[128.45167991472766, 38.44360845418272]",POINT (128.4516799147277 38.44360845418272),00000
1,라아907296,N/A,0.0,"POLYGON ((128.53675 38.26220, 128.53789 38.262...","[[128.53674989860596, 38.262195681986384], [12...","[128.53732774651786, 38.262641181764714]",POINT (128.5373277465179 38.26264118176471),00001
2,라아805562,39.00,39.0,"POLYGON ((128.42322 38.50286, 128.42436 38.502...","[[128.42321513343038, 38.50286417766996], [128...","[128.42379424556938, 38.50331020336737]",POINT (128.4237942455694 38.50331020336737),00002
3,라아918334,N/A,0.0,"POLYGON ((128.54981 38.29633, 128.55096 38.296...","[[128.54981456778376, 38.29632555548233], [128...","[128.55039276952658, 38.296770983303354]",POINT (128.5503927695266 38.29677098330335),00003
4,라아805526,N/A,0.0,"POLYGON ((128.42280 38.47042, 128.42395 38.470...","[[128.4228013214943, 38.47042465504401], [128....","[128.4233801674755, 38.47087068851492]",POINT (128.4233801674755 38.47087068851492),00004
...,...,...,...,...,...,...,...,...
71576,라아893177,,0.0,"POLYGON ((128.51925 38.15510, 128.52039 38.155...","[[128.51925287359862, 38.155103313580724], [12...","[128.51982975576533, 38.155548920969245]",POINT (128.5198297557653 38.15554892096925),71576
71577,라아857163,,0.0,"POLYGON ((128.47800 38.14284, 128.47914 38.142...","[[128.4780003347657, 38.14283679526627], [128....","[128.47857687210077, 38.14328261097056]",POINT (128.4785768721008 38.14328261097056),71577
71578,라아890218,,0.0,"POLYGON ((128.51634 38.19208, 128.51748 38.192...","[[128.51634225100835, 38.1920789478286], [128....","[128.51691941246253, 38.19252456289475]",POINT (128.5169194124625 38.19252456289475),71578
71579,라아854182,,0.0,"POLYGON ((128.47481 38.15999, 128.47595 38.159...","[[128.4748050967108, 38.15998683245551], [128....","[128.47538175223931, 38.16043266099766]",POINT (128.4753817522393 38.16043266099766),71579


In [27]:
%%time
# 최종 분석 데이터 정제하기

# 개발 가능 한 grid point 찾기
shapely.speedups.enable()
df_result = df_08[['gid','grid_id','val','geometry','coordinates','coord_cent','geo_cent']]
df_result['val'] = df_result['val'].fillna(0)
point_cent= gpd.GeoDataFrame(df_result[['grid_id','geo_cent']],geometry = 'geo_cent')

#굉장히 오래걸림
# within_points=point_cent.buffer(0.00000001).within(geo_possible.loc[0,'geometry'])
# pd.DataFrame(within_points).to_csv("within_points.csv", index = False)

within_points=pd.read_csv("within_points.csv")
df_result['개발가능'] = 0
df_result['개발가능'][within_points['0']==True] = 1
df_result[df_result['개발가능']==1]



## 71574개 중 9042개가 개발 가능한 구역

CPU times: user 128 ms, sys: 6.19 ms, total: 134 ms
Wall time: 423 ms


### 분석 결과
- 기본 상식선에서 개발 가능한 토지를 구분하였으나 토지관련 전문가를 통해 더 자세하게 토지를 추가/제거 한다면 더 현실적인 결과가 나올 것이다.

-------------------------------------

## 100X100 Point에 관광지 데이터 부여
- 관광지 geometry

In [28]:
df_21 = gpd.read_file('21.관광지_gemetry.GeoJSON')
df_22 = gpd.read_file('22.상권_gemetry.GeoJSON')

In [29]:
df_21

,score_stay,관광지_네비검색량,관광지_방문객수,정규화_관광지_score_stay,정규화_관광지_네비검색량,정규화_관광지_방문객수,관광지_동_유동인구,geometry
0,5.446506,74549,2.478197e+05,0.341691,0.010331,0.040114,85.826250,"POLYGON ((128.42322 38.50286, 128.42436 38.502..."
1,5.446506,8776,5.799000e+03,0.341691,0.001216,0.000939,85.826250,"POLYGON ((128.41878 38.51461, 128.41992 38.514..."
2,5.446506,8776,5.799000e+03,0.341691,0.001216,0.000939,154.064884,"POLYGON ((128.54539 38.22697, 128.54653 38.226..."
3,5.446506,74549,2.478197e+05,0.341691,0.010331,0.040114,91.602119,"POLYGON ((128.46975 38.38083, 128.47089 38.380..."
4,5.446506,74549,2.478197e+05,0.341691,0.010331,0.040114,77.015733,"POLYGON ((128.44567 38.46574, 128.44681 38.465..."
...,...,...,...,...,...,...,...,...
180,5.446506,74549,2.478197e+05,0.341691,0.010331,0.040114,439.385207,"POLYGON ((128.48982 38.17338, 128.49097 38.173..."
181,5.046835,144252,6.253920e+05,0.316618,0.019990,0.101231,75.606268,"POLYGON ((128.59437 38.21751, 128.59552 38.217..."
182,2.183564,3608062,3.088922e+06,0.136988,0.500000,0.500000,223.860765,"POLYGON ((128.52224 38.20555, 128.52338 38.205..."
183,5.245982,119223,3.928802e+05,0.329111,0.016522,0.063595,285.010999,"POLYGON ((128.57340 38.18977, 128.57455 38.189..."


In [30]:
%%time
# grid 마다 정규화_관광지_score_stay 부여
df_21_grid = []
df_superset = df_21

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_21_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_21_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_21_grid)):
    id_idx = df_21_grid[i][0]
    grids = df_21_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_21_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_21_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['정규화_관광지_score_stay'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['정규화_관광지_score_stay']
except:
    pass

grid_21=pd.DataFrame(grid_21_list)
grid_21.columns = ["grid_id","정규화_관광지_score_stay"]

df_result = pd.merge(df_result, grid_21, on = 'grid_id')
df_result


  0%|          | 0/185 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  185


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 3min 17s, sys: 6.39 s, total: 3min 24s
Wall time: 3min 18s


In [31]:
%%time
# grid 마다 정규화_관광지_네비검색량 부여
df_21_grid = []
df_superset = df_21

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_21_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_21_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_21_grid)):
    id_idx = df_21_grid[i][0]
    grids = df_21_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_21_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_21_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['정규화_관광지_네비검색량'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['정규화_관광지_네비검색량']
except:
    pass

grid_21=pd.DataFrame(grid_21_list)
grid_21.columns = ["grid_id","정규화_관광지_네비검색량"]

df_result = pd.merge(df_result, grid_21, on = 'grid_id')
df_result

  0%|          | 0/185 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  185


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 3min 10s, sys: 6.04 s, total: 3min 16s
Wall time: 3min 10s


In [32]:
%%time
# grid 마다 정규화_관광지_방문객수 부여
df_21_grid = []
df_superset = df_21

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_21_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_21_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_21_grid)):
    id_idx = df_21_grid[i][0]
    grids = df_21_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_21_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_21_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['정규화_관광지_방문객수'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['정규화_관광지_방문객수']
except:
    pass

grid_21=pd.DataFrame(grid_21_list)
grid_21.columns = ["grid_id","정규화_관광지_방문객수"]

df_result = pd.merge(df_result, grid_21, on = 'grid_id')
df_result

  0%|          | 0/185 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  185


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 3min 11s, sys: 6.06 s, total: 3min 17s
Wall time: 3min 10s


In [33]:
%%time
# grid 마다 관광지_동_유동인구 부여
df_21_grid = []
df_superset = df_21

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_21_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_21_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_21_grid)):
    id_idx = df_21_grid[i][0]
    grids = df_21_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_21_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_21_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['관광지_동_유동인구'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['관광지_동_유동인구']
except:
    pass

grid_21=pd.DataFrame(grid_21_list)
grid_21.columns = ["grid_id","관광지_동_유동인구"]

df_result = pd.merge(df_result, grid_21, on = 'grid_id')
df_result


  0%|          | 0/185 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  185


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 3min 11s, sys: 6.36 s, total: 3min 18s
Wall time: 3min 10s


## 100X100 Point에 상권 데이터 부여
- 상권 geometry

In [34]:
%%time
# grid 마다 정규화_상권_score_search_point 부여
df_22_grid = []
df_superset = df_22

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_22_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_22_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_22_grid)):
    id_idx = df_22_grid[i][0]
    grids = df_22_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_22_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_22_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['정규화_상권_score_search_point'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['정규화_상권_score_search_point']
except:
    pass

grid_22=pd.DataFrame(grid_22_list)
grid_22.columns = ["grid_id","정규화_상권_score_search_point"]

df_result = pd.merge(df_result, grid_22, on = 'grid_id')
df_result

  0%|          | 0/1236 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  1236


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 8min 6s, sys: 7.54 s, total: 8min 14s
Wall time: 8min 8s


In [35]:
%%time
# grid 마다 정규화_상권_score_view_count 부여
df_22_grid = []
df_superset = df_22

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_22_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_22_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_22_grid)):
    id_idx = df_22_grid[i][0]
    grids = df_22_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_22_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_22_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['정규화_상권_score_view_count'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['정규화_상권_score_view_count']
except:
    pass

grid_22=pd.DataFrame(grid_22_list)
grid_22.columns = ["grid_id","정규화_상권_score_view_count"]

df_result = pd.merge(df_result, grid_22, on = 'grid_id')
df_result

  0%|          | 0/1236 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  1236


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 8min 7s, sys: 7.56 s, total: 8min 15s
Wall time: 8min 9s


In [36]:
%%time
# grid 마다 정규화_상권_score_review_count 부여
df_22_grid = []
df_superset = df_22

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_22_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_22_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_22_grid)):
    id_idx = df_22_grid[i][0]
    grids = df_22_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_22_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_22_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['정규화_상권_score_review_count'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['정규화_상권_score_review_count']
except:
    pass

grid_22=pd.DataFrame(grid_22_list)
grid_22.columns = ["grid_id","정규화_상권_score_review_count"]

df_result = pd.merge(df_result, grid_22, on = 'grid_id')
df_result


  0%|          | 0/1236 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  1236


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 8min 9s, sys: 8.03 s, total: 8min 17s
Wall time: 8min 10s


In [37]:
df_22

,score_search_point,score_view_count,score_review_count,score_stay,정규화_상권_score_search_point,정규화_상권_score_view_count,정규화_상권_score_review_count,정규화_상권_score_stay,상권_동_유동인구,geometry,grid_ids
0,30.0,11.446266,7.355742,4.666667,0.020619,0.025784,0.017504,0.020558,308.129767,"POLYGON ((128.51093 38.21376, 128.51207 38.213...","('00008', '00472', '00644', '04304', '04821', ..."
1,30.0,7.446266,9.355742,4.666667,0.020619,0.016773,0.022263,0.020558,308.129767,"POLYGON ((128.50742 38.20748, 128.50856 38.207...","('00010', '02226', '04083', '06046', '11789', ..."
2,30.0,8.892532,8.711485,4.666667,0.020619,0.020031,0.020730,0.020558,197.605433,"POLYGON ((128.50673 38.32284, 128.50787 38.322...","('00013', '00331', '00777', '02254', '02335', ..."
3,15.0,4.446266,4.355742,2.333333,0.010309,0.010016,0.010365,0.010279,98.802717,"POLYGON ((128.53620 38.30366, 128.53734 38.303...","('00019', '00910', '01045', '01287', '02359', ..."
4,60.0,16.338798,18.067227,9.333333,0.041237,0.036805,0.042993,0.041116,366.408475,"POLYGON ((128.46857 38.37813, 128.46971 38.378...","('00029', '00128', '00223', '00417', '00630', ..."
...,...,...,...,...,...,...,...,...,...,...,...
1231,60.0,17.785064,17.422969,9.333333,0.041237,0.040063,0.041460,0.041116,570.021997,"POLYGON ((128.59056 38.19141, 128.59170 38.191...","('60934', '61146', '61234', '65872', '65926', ..."
1232,15.0,4.446266,4.355742,20.000000,0.010309,0.010016,0.010365,0.088106,109.846302,"POLYGON ((128.51375 38.16957, 128.51489 38.169...","('60907', '61892', '62511', '63170', '68219', ..."
1233,30.0,8.892532,8.711485,4.666667,0.020619,0.020031,0.020730,0.020558,79.735543,"POLYGON ((128.59602 38.17514, 128.59716 38.175...","('61238', '61820', '62390', '66609', '67685', ..."
1234,15.0,10.000000,7.000000,2.333333,0.010309,0.022526,0.016657,0.010279,285.010999,"POLYGON ((128.57340 38.18977, 128.57455 38.189...","('60627', '60645', '60851', '61037', '61354', ..."


In [38]:
%%time
# grid 마다 정규화_상권_score_stay 부여
df_22_grid = []
df_superset = df_22

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_22_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_22_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_22_grid)):
    id_idx = df_22_grid[i][0]
    grids = df_22_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_22_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_22_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['정규화_상권_score_stay'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['정규화_상권_score_stay']
except:
    pass

grid_22=pd.DataFrame(grid_22_list)
grid_22.columns = ["grid_id","정규화_상권_score_stay"]

df_result = pd.merge(df_result, grid_22, on = 'grid_id')
df_result

  0%|          | 0/1236 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  1236


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 8min 7s, sys: 7.73 s, total: 8min 15s
Wall time: 8min 8s


In [39]:
%%time
# grid 마다 정규화_상권_score_stay 부여
df_22_grid = []
df_superset = df_22

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_22_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_22_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_22_grid)):
    id_idx = df_22_grid[i][0]
    grids = df_22_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_22_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_22_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['상권_동_유동인구'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['상권_동_유동인구']
except:
    pass

grid_22=pd.DataFrame(grid_22_list)
grid_22.columns = ["grid_id","상권_동_유동인구"]

df_result = pd.merge(df_result, grid_22, on = 'grid_id')
df_result


  0%|          | 0/1236 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  1236


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 8min 6s, sys: 7.27 s, total: 8min 13s
Wall time: 8min 7s


## 100X100 Point에 교통량, 혼잡빈도강도, 혼잡시간강도,   관련 요소 부여

**목적: grid 마다 교통량 관련 요소 부여**

**분석 데이터 종류**
- df_11: 11.평일_일별_시간대별__추정교통량.csv
- df_12: 12.평일_혼잡빈도강도_강원도 속초시, 고성군.csv
- df_13: 13.평일_혼잡시간강도_강원도 속초시, 고성군.csv

**분석 설명**
- 각 100X100 Point 마다 11시 교통량, 14시 교통량, 혼잡빈도강도합, 혼잡시간강도합을 부여
- 각 요소바다 부여하는데 시간이 다소 소요됨 (약 10분)

In [40]:
%%time
# grid 마다 11시 교통량 부여
df_10_11_time11_grid = []
df_superset = df_10_11_time11[df_10_11_time11['교통량']>0]

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_10_11_time11_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_10_11_time11_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_10_11_time11_grid)):
    id_idx = df_10_11_time11_grid[i][0]
    grids = df_10_11_time11_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['교통량'])])
    except:
        pass

#2시 승용차 혼잡 빈도 관련 정보
try:
    del df_result['교통량_11']
except:
    pass

grid_=pd.DataFrame(grid_list)
grid_.columns = ["grid_id","교통량_11"]
#grid_혼잡빈도[grid_혼잡빈도['승용차_혼잡빈도강도합']>0]
df_result = pd.merge(df_result, grid_, on = 'grid_id')
df_result[df_result['교통량_11']>0]


## 교통량_11 2798개 중 2381개가 point와 관련되어있는 grid 개수이고
## 임의 gird 중 71581개 중 교통량_11 5754개가 개발가능여부 부여

  0%|          | 0/2798 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  2381


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 15min 7s, sys: 13 s, total: 15min 20s
Wall time: 15min 10s


In [41]:
%%time
# grid 마다 14시 교통량 부여
df_10_11_time14_grid = []
df_superset = df_10_11_time14[df_10_11_time14['교통량']>0]

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_10_11_time14_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_10_11_time14_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_10_11_time14_grid)):
    id_idx = df_10_11_time14_grid[i][0]
    grids = df_10_11_time14_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_혼잡빈도_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_혼잡빈도_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['교통량'])])
    except:
        pass

grid_혼잡빈도=pd.DataFrame(grid_혼잡빈도_list)
grid_혼잡빈도.columns = ["grid_id","교통량_14"]
grid_혼잡빈도[grid_혼잡빈도['교통량_14']>0]


#17시 승용차 혼잡 빈도 관련 정보
try:
    del df_result['교통량_14']
except:
    pass

grid_혼잡빈도=pd.DataFrame(grid_혼잡빈도_list)
grid_혼잡빈도.columns = ["grid_id","교통량_14"]
#grid_혼잡빈도[grid_혼잡빈도['승용차_혼잡빈도강도합']>0]
df_result = pd.merge(df_result, grid_혼잡빈도, on = 'grid_id')
df_result[df_result['교통량_14']>0]




## 교통량_17 2559개 중 2079개가 point와 관련되어있는 grid 개수이고
## 임의 gird 중 71581개 중 교통량_17 4743개가 개발가능여부 부여

  0%|          | 0/2612 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  2116


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 14min 2s, sys: 11.9 s, total: 14min 14s
Wall time: 14min 17s


In [42]:
%%time
# grid 마다 혼잡빈도강도 부여
df_10_grid = []
df_superset = df_10_12[df_10_12['혼잡빈도강도합']>0]

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_10_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_10_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_10_grid)):
    id_idx = df_10_grid[i][0]
    grids = df_10_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_혼잡빈도_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_혼잡빈도_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['혼잡빈도강도합'])])
    except:
        pass

#혼잡빈도강도 관련 정보
try:
    del df_result['혼잡빈도강도합']
except:
    pass

grid_혼잡빈도=pd.DataFrame(grid_혼잡빈도_list)
grid_혼잡빈도.columns = ["grid_id","혼잡빈도강도합"]
#grid_혼잡빈도[grid_혼잡빈도['승용차_혼잡빈도강도합']>0]
df_result = pd.merge(df_result, grid_혼잡빈도, on = 'grid_id')
df_result[df_result['혼잡빈도강도합']>0]



## 혼잡빈도강도합 2878개 중 2507개가 point와 관련되어있는 grid 개수이고
## 임의 gird 중 71581개 중 혼잡빈도강도합 6624개가 개발가능여부 부여

  0%|          | 0/2878 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  2507


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 15min 53s, sys: 13.4 s, total: 16min 7s
Wall time: 15min 57s


In [43]:
%%time
# grid 마다 혼잡시간강도합 부여
df_10_grid = []
df_superset = df_10_13[df_10_13['혼잡시간강도합']>0]

#시간이 오래 걸립니다.
for i in tqdm(range(len(df_superset))): #len(df_superset)
    try:
        grid_ids = point_cent[point_cent.within(df_superset.loc[i,'geometry'].buffer(0.001))]['grid_id']
        if len(grid_ids) != 0:
            df_10_grid.append([i,str(tuple(grid_ids))])
    except :
        pass

print('Point와 관련된 grid 개수: ',len(df_10_grid))

df_superset['grid_ids'] = 0
for i in range(len(df_10_grid)):
    id_idx = df_10_grid[i][0]
    grids = df_10_grid[i][1]
    df_superset['grid_ids'][id_idx] = grids


#시간이 오래 걸립니다.
grid_혼잡빈도_list = []
for i in tqdm(df_result['grid_id']):
    try:
        grid_혼잡빈도_list.append([i, sum(df_superset[df_superset['grid_ids'].str.contains(i)==True]['혼잡시간강도합'])])
    except:
        pass

#혼잡시간강도합 관련 정보
try:
    del df_result['혼잡시간강도합']
except:
    pass

grid_혼잡빈도=pd.DataFrame(grid_혼잡빈도_list)
grid_혼잡빈도.columns = ["grid_id","혼잡시간강도합"]
#grid_혼잡빈도[grid_혼잡빈도['승용차_혼잡빈도강도합']>0]
df_result = pd.merge(df_result, grid_혼잡빈도, on = 'grid_id')
df_result[df_result['혼잡시간강도합']>0]

## 혼잡시간강도합 2904개 중 2555개가 point와 관련되어있는 grid 개수이고
## 임의 gird 중 71581개 중 혼잡시간강도 6861개가 개발가능여부

  0%|          | 0/2904 [00:00<?, ?it/s]

Point와 관련된 grid 개수:  2555


  0%|          | 0/71581 [00:00<?, ?it/s]

CPU times: user 16min 11s, sys: 13.6 s, total: 16min 25s
Wall time: 16min 15s


---------------------------------
## 기존 충전소 위치 분석


**목적: 기존 충전소가 있는 위치를 분석, 기존 충전소가 커버가능한 범위는 제외하고 분석**

**분석 데이터 종류**
- df_01: 01.고성군_속초시_충전기설치현황.csv

**분석 설명**
- 급속 충전소 (Fast-charing Station, FS) 와 완속 충전소(Slow-charging Station, SS) 의 위치를 확인하였다.
    - 급속: 파란색
    - 완속: 초록색
- 급속 충전소와 완속 충전소 주위 500m를 cover가능하다고 가정하였다.
- 기존 충전소가 cover 가능한 point를 구분하였다.

In [44]:
df_01 = pd.read_csv('01.고성군_속초시_충전기설치현황.csv')

In [45]:
%%time
# 기존 완속/ 급속 충전소가 커버하는 위치 제거
df_01_geo = []
for i in range(len(df_01)):
    df_01_geo.append([df_01.loc[i,'충전소명'],Point(df_01.loc[i,'lon'],df_01.loc[i,'lat']).buffer(0.003)])
# df_01[df_01['급속/완속']=='완속']
df_01_geo = pd.DataFrame(df_01_geo)
df_01_geo.columns = ["충전소명", "geometry"]
df_01_geo = pd.merge(df_01, df_01_geo, on = '충전소명')
df_01_geo['coordinates'] = df_01_geo['geometry'].apply(polygon_to_coordinates) 
df_01_geo = pd.DataFrame(df_01_geo)





center = [128.5918, 38.27701] # 센터 [128.5918, 38.27701]
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 
layer1 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_01_geo[df_01_geo['급속/완속']=='급속'][['coordinates']], # 시각화에 쓰일 데이터프레임
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[50, 50, 200,140]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 

layer2 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_01_geo[df_01_geo['급속/완속']=='완속'][['coordinates']], # 시각화에 쓰일 데이터프레임
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[100, 200, 100,140]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 

scatt1 = pdk.Layer(
    'ScatterplotLayer',
    df_01_geo[df_01_geo['급속/완속']=='급속'][['lon','lat']],
    get_position = ['lon','lat'],
    auto_highlight=True,
    get_radius=50,
    get_fill_color='[50, 50, 200]',
    pickable=True)

scatt2 = pdk.Layer(
    'ScatterplotLayer',
    df_01_geo[df_01_geo['급속/완속']=='완속'][['lon','lat']],
    get_position = ['lon','lat'],
    auto_highlight=True,
    get_radius=50,
    get_fill_color='[100, 200, 100]',
    pickable=True)


r = pdk.Deck(layers=[layer1,scatt1, layer2,scatt2], initial_view_state=view_state)
   
r.to_html()

<IPython.core.display.Javascript object>

CPU times: user 75.7 ms, sys: 6.09 ms, total: 81.8 ms
Wall time: 89.1 ms


In [46]:
#Fast-charging Station

#시간이 많이 걸립니다.
df_01_FS = df_01_geo[df_01_geo['급속/완속']=='급속']
FS_points = []
for i in tqdm(range(len(df_01_FS))):
    try:
        FS_points.append(point_cent.buffer(0.00000001).within(df_01_FS.loc[i,'geometry']))
    except: 
        pass
df_result['FS_station'] = 0
for i in range(len(FS_points)):
    df_result['FS_station'][FS_points[i]] = 1

#Slow-charging Station
df_01_SS = df_01_geo[df_01_geo['급속/완속']=='완속']    
SS_points = [] 
for i in tqdm(range(len(df_01_geo))):
    try:
        SS_points.append(point_cent.buffer(0.00000001).within(df_01_SS.loc[i,'geometry']))
    except:
        pass

df_result['SS_station'] = 0
for i in range(len(SS_points)):
    df_result['SS_station'][SS_points[i]] = 1

df_result.head()

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

,gid,grid_id,val,geometry,coordinates,coord_cent,geo_cent,개발가능,정규화_관광지_score_stay,정규화_관광지_네비검색량,정규화_관광지_방문객수,관광지_동_유동인구,정규화_상권_score_search_point,정규화_상권_score_view_count,정규화_상권_score_review_count,정규화_상권_score_stay,상권_동_유동인구,교통량_11,교통량_14,혼잡빈도강도합,혼잡시간강도합,FS_station,SS_station
0,라아830496,00000,30.0,"POLYGON ((128.45110 38.44316, 128.45225 38.443...","[[128.4511011187016, 38.4431625579364], [128.4...","[128.45167991472766, 38.44360845418272]",POINT (128.4516799147277 38.44360845418272),1,0.000000,0.000000,0.000000,0.00000,0.103093,0.096898,0.100423,0.180617,770.157331,0.0,0.0,0,0,0,0
1,라아907296,00001,0.0,"POLYGON ((128.53675 38.26220, 128.53789 38.262...","[[128.53674989860596, 38.262195681986384], [12...","[128.53732774651786, 38.262641181764714]",POINT (128.5373277465179 38.26264118176471),0,0.000000,0.000000,0.000000,0.00000,0.010309,0.010016,0.010365,0.088106,308.129767,0.0,0.0,5,16,0,0
2,라아805562,00002,39.0,"POLYGON ((128.42322 38.50286, 128.42436 38.502...","[[128.42321513343038, 38.50286417766996], [128...","[128.42379424556938, 38.50331020336737]",POINT (128.4237942455694 38.50331020336737),0,0.341691,0.010331,0.040114,85.82625,0.010309,0.010016,0.010365,0.010279,85.826250,0.0,0.0,73,134,0,0
3,라아918334,00003,0.0,"POLYGON ((128.54981 38.29633, 128.55096 38.296...","[[128.54981456778376, 38.29632555548233], [128...","[128.55039276952658, 38.296770983303354]",POINT (128.5503927695266 38.29677098330335),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0,0,0
4,라아805526,00004,0.0,"POLYGON ((128.42280 38.47042, 128.42395 38.470...","[[128.4228013214943, 38.47042465504401], [128....","[128.4233801674755, 38.47087068851492]",POINT (128.4233801674755 38.47087068851492),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0,0,0


In [47]:
df_result.columns

Index(['gid', 'grid_id', 'val', 'geometry', 'coordinates', 'coord_cent', 'geo_cent', '개발가능', '정규화_관광지_score_stay', '정규화_관광지_네비검색량', '정규화_관광지_방문객수', '관광지_동_유동인구', '정규화_상권_score_search_point', '정규화_상권_score_view_count', '정규화_상권_score_review_count', '정규화_상권_score_stay', '상권_동_유동인구', '교통량_11', '교통량_14', '혼잡빈도강도합', '혼잡시간강도합', 'FS_station', 'SS_station'], dtype='object')

### 분석 결과
- 전기자동차 충전소가 cover가능한 거리는 임의로 정한 것으로 바뀔 필요가 있다.
- 현재는 전기자동차가 많이 등록되어 있지 않아 거리로 임의로 정하였지만 대수가 많아 진다면 전기자동차 등록 대수에 따라 cover 가능한 거리가 바뀌어야 할 것이다.
- 즉, 전기자동차 등록이 많은 곳은 cover 가능한 거리를 줄여 더 많은 곳에 충전소를 설치해야 한다.
-------------------------------------



---------------------------------
## 전기자동차 충전소 위치선정에 대한 영향 요소 분석 및 상관관계 분석

본 지원자는 전기자동차 충전소 위치 선정을 최적화 문제로 풀 것이다. 이를 위해 목적함수가 필요하며 다음과 같은 기준으로 식을 세웠다. 

**가정**
1. 전기자동차 충전소 위치는 인구현황, 교통량, 혼잡빈도강도, 혼잡시간강도 만 고려하여 위치를 선정한다.
2. 기존 설치된 전기자동차 충전소는 위 고려사항을 충분히 고려하여 만들어진 곳이다.
3. 전기자동차 충전소는 전방 약 500m를 커버할 수 있다. 


**분석 방법**

- 고려되는 모든 변수들은 정규화 하였다.
- 선형회귀분석을 이용해 현재 제공받은 데이터로부터 전기자동차 충전소 위치에 영향을 주는 요소의 관계를 분석하였다.(Linear Regrssion)

- 이때 급속 충전소와 완속 충전소 각각을 따로 분석하였다.  

- **분석 Input**: (정규화된) 인구현황, 11시 교통량, 14시 교통량, 혼잡빈도강도, 혼잡시간강도

- **분석 Output**: 고려되는 요소들과 각 충전소 사이의 상관계수 



In [48]:
df_21.columns

Index(['score_stay', '관광지_네비검색량', '관광지_방문객수', '정규화_관광지_score_stay', '정규화_관광지_네비검색량', '정규화_관광지_방문객수', '관광지_동_유동인구', 'geometry', 'grid_ids'], dtype='object')

In [49]:
df_22.columns

Index(['score_search_point', 'score_view_count', 'score_review_count', 'score_stay', '정규화_상권_score_search_point', '정규화_상권_score_view_count', '정규화_상권_score_review_count', '정규화_상권_score_stay', '상권_동_유동인구', 'geometry', 'grid_ids'], dtype='object')

In [50]:
## 상권, 교통량 혼잡강도 Linear Regresiion 비교 설정

df_result['정규화_교통량_11'] = df_result['교통량_11'] / df_result['교통량_11'].max()
df_result['정규화_교통량_14'] = df_result['교통량_14'] / df_result['교통량_14'].max()
df_result['정규화_혼잡빈도강도합'] = df_result['혼잡빈도강도합'] / df_result['혼잡빈도강도합'].max()
df_result['정규화_혼잡시간강도합'] = df_result['혼잡시간강도합'] / df_result['혼잡시간강도합'].max()
df_result['정규화_관광지_동_유동인구'] = df_result['관광지_동_유동인구'] / df_result['관광지_동_유동인구'].max()
df_result['정규화_상권_동_유동인구'] = df_result['상권_동_유동인구'] / df_result['상권_동_유동인구'].max()



# 급속/ 완속 관련 objective function 만들기
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

df_LR = df_result
X = df_LR[["정규화_교통량_11","정규화_교통량_14","정규화_혼잡빈도강도합","정규화_혼잡시간강도합",
           '정규화_관광지_score_stay', '정규화_관광지_네비검색량', '정규화_관광지_방문객수', '정규화_관광지_동_유동인구',
           '정규화_상권_score_search_point', '정규화_상권_score_view_count',
           '정규화_상권_score_review_count', '정규화_상권_score_stay', '정규화_상권_동_유동인구']]

y = df_LR["FS_station"]
regr = linear_model.LinearRegression()
regr.fit(X, y)
FS_coeff = regr.coef_
print('급속충전소 Intercept: ', regr.intercept_)
print('급속충전소 Coefficients: \n', FS_coeff)

df_LR = df_result
X = df_LR[["정규화_교통량_11","정규화_교통량_14","정규화_혼잡빈도강도합","정규화_혼잡시간강도합",
           '정규화_관광지_score_stay', '정규화_관광지_네비검색량', '정규화_관광지_방문객수', '정규화_관광지_동_유동인구',
           '정규화_상권_score_search_point', '정규화_상권_score_view_count',
           '정규화_상권_score_review_count', '정규화_상권_score_stay', '정규화_상권_동_유동인구']]
y = df_LR["SS_station"]
regr = linear_model.LinearRegression()
regr.fit(X, y)
SS_coeff = regr.coef_
print('완속충전소 Intercept: ', regr.intercept_)
print('완속충전소 Coefficients: \n', SS_coeff)

급속충전소 Intercept:  0.00326511191804562
급속충전소 Coefficients: 
 [-0.18019568  0.04908327 -1.08969476  1.38152381  0.10248282 -0.17466468
 -0.03841347 -0.0307803  -0.4524974   3.44420118 -2.34795205 -0.08388847
 -0.67096205]
완속충전소 Intercept:  0.007119322017588053
완속충전소 Coefficients: 
 [-1.07108613  1.001021    0.38282749  0.596246    0.17295541 -0.10968143
 -0.00654608 -2.33804435 -3.08426437 -1.61592229  4.73564699  0.23300885
  0.99028133]


In [51]:
df_result

,gid,grid_id,val,geometry,coordinates,coord_cent,geo_cent,개발가능,정규화_관광지_score_stay,정규화_관광지_네비검색량,정규화_관광지_방문객수,관광지_동_유동인구,정규화_상권_score_search_point,정규화_상권_score_view_count,정규화_상권_score_review_count,정규화_상권_score_stay,상권_동_유동인구,교통량_11,교통량_14,혼잡빈도강도합,혼잡시간강도합,FS_station,SS_station,정규화_교통량_11,정규화_교통량_14,정규화_혼잡빈도강도합,정규화_혼잡시간강도합,정규화_관광지_동_유동인구,정규화_상권_동_유동인구
0,라아830496,00000,30.0,"POLYGON ((128.45110 38.44316, 128.45225 38.443...","[[128.4511011187016, 38.4431625579364], [128.4...","[128.45167991472766, 38.44360845418272]",POINT (128.4516799147277 38.44360845418272),1,0.000000,0.000000,0.000000,0.00000,0.103093,0.096898,0.100423,0.180617,770.157331,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.023149
1,라아907296,00001,0.0,"POLYGON ((128.53675 38.26220, 128.53789 38.262...","[[128.53674989860596, 38.262195681986384], [12...","[128.53732774651786, 38.262641181764714]",POINT (128.5373277465179 38.26264118176471),0,0.000000,0.000000,0.000000,0.00000,0.010309,0.010016,0.010365,0.088106,308.129767,0.00,0.00,5,16,0,0,0.000000,0.000000,0.001497,0.004388,0.000000,0.009262
2,라아805562,00002,39.0,"POLYGON ((128.42322 38.50286, 128.42436 38.502...","[[128.42321513343038, 38.50286417766996], [128...","[128.42379424556938, 38.50331020336737]",POINT (128.4237942455694 38.50331020336737),0,0.341691,0.010331,0.040114,85.82625,0.010309,0.010016,0.010365,0.010279,85.826250,0.00,0.00,73,134,0,0,0.000000,0.000000,0.021856,0.036753,0.006974,0.002580
3,라아918334,00003,0.0,"POLYGON ((128.54981 38.29633, 128.55096 38.296...","[[128.54981456778376, 38.29632555548233], [128...","[128.55039276952658, 38.296770983303354]",POINT (128.5503927695266 38.29677098330335),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,라아805526,00004,0.0,"POLYGON ((128.42280 38.47042, 128.42395 38.470...","[[128.4228013214943, 38.47042465504401], [128....","[128.4233801674755, 38.47087068851492]",POINT (128.4233801674755 38.47087068851492),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71576,라아893177,71576,0.0,"POLYGON ((128.51925 38.15510, 128.52039 38.155...","[[128.51925287359862, 38.155103313580724], [12...","[128.51982975576533, 38.155548920969245]",POINT (128.5198297557653 38.15554892096925),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
71577,라아857163,71577,0.0,"POLYGON ((128.47800 38.14284, 128.47914 38.142...","[[128.4780003347657, 38.14283679526627], [128....","[128.47857687210077, 38.14328261097056]",POINT (128.4785768721008 38.14328261097056),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
71578,라아890218,71578,0.0,"POLYGON ((128.51634 38.19208, 128.51748 38.192...","[[128.51634225100835, 38.1920789478286], [128....","[128.51691941246253, 38.19252456289475]",POINT (128.5169194124625 38.19252456289475),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
71579,라아854182,71579,0.0,"POLYGON ((128.47481 38.15999, 128.47595 38.159...","[[128.4748050967108, 38.15998683245551], [128....","[128.47538175223931, 38.16043266099766]",POINT (128.4753817522393 38.16043266099766),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [52]:
df_result.columns

Index(['gid', 'grid_id', 'val', 'geometry', 'coordinates', 'coord_cent', 'geo_cent', '개발가능', '정규화_관광지_score_stay', '정규화_관광지_네비검색량', '정규화_관광지_방문객수', '관광지_동_유동인구', '정규화_상권_score_search_point', '정규화_상권_score_view_count', '정규화_상권_score_review_count', '정규화_상권_score_stay', '상권_동_유동인구', '교통량_11', '교통량_14', '혼잡빈도강도합', '혼잡시간강도합', 'FS_station', 'SS_station', '정규화_교통량_11', '정규화_교통량_14', '정규화_혼잡빈도강도합', '정규화_혼잡시간강도합', '정규화_관광지_동_유동인구', '정규화_상권_동_유동인구'], dtype='object')

In [53]:
# OLS Regression Results 
import statsmodels.api as sm
X = sm.add_constant(X, has_constant='add')
modeling = sm.OLS(y, X)
modeling.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SS_station   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     1547.
Date:                Tue, 25 Jan 2022   Prob (F-statistic):               0.00
Time:                        16:07:27   Log-Likelihood:                 52401.
No. Observations:               71581   AIC:                        -1.048e+05
Df Residuals:                   71567   BIC:                        -1.046e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.0071      0.000     15.991      0.000       0.006       0.008
정규화_교통량_11                   -1.0711      0.144     -7.418      0.000      -1.354      -0.788
정규화_교통량_14                    1.0010      0.141      7.114      0.000       0.725       1.277
정규화_혼잡빈도강도합                   0.3828      0.096      3.992      0.000       0.195       0.571
정규화_혼잡시간강도합                   0.5962      0.091      6.572      0.000       0.418       0.774
정규화_관광지_score_stay            0.1730      0.009     19.199      0.000       0.155       0.191
정규화_관광지_네비검색량                -0.1097      0.027     -3.990      0.000      -0.164      -0.056
정규화_관광지_방문객수                 -0.0065      0.016     -0.415      0.678      -0.037       0.024
정규화_관광지_동_유동인구               -2.3380      0.073    -32.031      0.000      -2.481      -2.195
정규화_상권_score_search_point    -3.0843      0.312     -9.880      0.000      -3.696      -2.472
정규화_상권_score_view_count      -1.6159      0.254     -6.368      0.000      -2.113      -1.119
정규화_상권_score_review_count     4.7356      0.360     13.145      0.000       4.030       5.442
정규화_상권_score_stay             0.2330      0.011     21.580      0.000       0.212       0.254
정규화_상권_동_유동인구                 0.9903      0.076     13.102      0.000       0.842       1.138
==============================================================================
Omnibus:                    80976.018   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8463544.433
Skew:                           5.915   Prob(JB):                         0.00
Kurtosis:                      54.940   Cond. No.                     1.04e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
df_result['w_FS'] = 0 
df_result['w_FS'] = (FS_coeff[0]*df_result['정규화_교통량_11']+
                     FS_coeff[1]*df_result['정규화_교통량_14']+
                     FS_coeff[2]*df_result['정규화_혼잡빈도강도합']+
                     FS_coeff[2]*df_result['정규화_혼잡시간강도합']+
                     FS_coeff[3]*df_result['정규화_관광지_score_stay']+
                     FS_coeff[3]*df_result['정규화_관광지_네비검색량']+
                     FS_coeff[3]*df_result['정규화_관광지_방문객수']+
                     FS_coeff[3]*df_result['정규화_관광지_동_유동인구']+
                     FS_coeff[4]*df_result['정규화_상권_score_search_point']+                     
                     FS_coeff[4]*df_result['정규화_상권_score_view_count']+
                     FS_coeff[4]*df_result['정규화_상권_score_review_count']+
                     FS_coeff[4]*df_result['정규화_상권_score_stay']+
                     FS_coeff[4]*df_result['정규화_상권_동_유동인구']
                    )
df_result['w_SS'] = 0 
df_result['w_SS'] = (SS_coeff[0]*df_result['정규화_교통량_11']+
                     SS_coeff[1]*df_result['정규화_교통량_14']+
                     SS_coeff[2]*df_result['정규화_혼잡빈도강도합']+
                     SS_coeff[2]*df_result['정규화_혼잡시간강도합']+
                     SS_coeff[3]*df_result['정규화_관광지_score_stay']+
                     SS_coeff[3]*df_result['정규화_관광지_네비검색량']+
                     SS_coeff[3]*df_result['정규화_관광지_방문객수']+
                     SS_coeff[3]*df_result['정규화_관광지_동_유동인구']+
                     SS_coeff[4]*df_result['정규화_상권_score_search_point']+                     
                     SS_coeff[4]*df_result['정규화_상권_score_view_count']+
                     SS_coeff[4]*df_result['정규화_상권_score_review_count']+
                     SS_coeff[4]*df_result['정규화_상권_score_stay']+
                     SS_coeff[4]*df_result['정규화_상권_동_유동인구']
                    )

In [55]:
try:    
    df_result[['grid_id','geometry',
               '정규화_교통량_11','정규화_교통량_14',
               '정규화_혼잡빈도강도합', '정규화_혼잡시간강도합','정규화_관광지_score_stay','정규화_관광지_네비검색량',
               '정규화_관광지_방문객수','정규화_관광지_동_유동인구','정규화_상권_score_search_point',
               '정규화_상권_score_view_count','정규화_상권_score_review_count','정규화_상권_score_stay',
               '정규화_상권_동_유동인구',
               'w_FS','w_SS','개발가능','FS_station','SS_station']].to_file("df_result.geojson", driver="GeoJSON")
except:
    pass

In [56]:
df_result

,gid,grid_id,val,geometry,coordinates,coord_cent,geo_cent,개발가능,정규화_관광지_score_stay,정규화_관광지_네비검색량,정규화_관광지_방문객수,관광지_동_유동인구,정규화_상권_score_search_point,정규화_상권_score_view_count,정규화_상권_score_review_count,정규화_상권_score_stay,상권_동_유동인구,교통량_11,교통량_14,혼잡빈도강도합,혼잡시간강도합,FS_station,SS_station,정규화_교통량_11,정규화_교통량_14,정규화_혼잡빈도강도합,정규화_혼잡시간강도합,정규화_관광지_동_유동인구,정규화_상권_동_유동인구,w_FS,w_SS
0,라아830496,00000,30.0,"POLYGON ((128.45110 38.44316, 128.45225 38.443...","[[128.4511011187016, 38.4431625579364], [128.4...","[128.45167991472766, 38.44360845418272]",POINT (128.4516799147277 38.44360845418272),1,0.000000,0.000000,0.000000,0.00000,0.103093,0.096898,0.100423,0.180617,770.157331,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.023149,0.051670,0.087201
1,라아907296,00001,0.0,"POLYGON ((128.53675 38.26220, 128.53789 38.262...","[[128.53674989860596, 38.262195681986384], [12...","[128.53732774651786, 38.262641181764714]",POINT (128.5373277465179 38.26264118176471),0,0.000000,0.000000,0.000000,0.00000,0.010309,0.010016,0.010365,0.088106,308.129767,0.00,0.00,5,16,0,0,0.000000,0.000000,0.001497,0.004388,0.000000,0.009262,0.006710,0.024401
2,라아805562,00002,39.0,"POLYGON ((128.42322 38.50286, 128.42436 38.502...","[[128.42321513343038, 38.50286417766996], [128...","[128.42379424556938, 38.50331020336737]",POINT (128.4237942455694 38.50331020336737),0,0.341691,0.010331,0.040114,85.82625,0.010309,0.010016,0.010365,0.010279,85.826250,0.00,0.00,73,134,0,0,0.000000,0.000000,0.021856,0.036753,0.006974,0.002580,0.491978,0.267937
3,라아918334,00003,0.0,"POLYGON ((128.54981 38.29633, 128.55096 38.296...","[[128.54981456778376, 38.29632555548233], [128...","[128.55039276952658, 38.296770983303354]",POINT (128.5503927695266 38.29677098330335),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,라아805526,00004,0.0,"POLYGON ((128.42280 38.47042, 128.42395 38.470...","[[128.4228013214943, 38.47042465504401], [128....","[128.4233801674755, 38.47087068851492]",POINT (128.4233801674755 38.47087068851492),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71576,라아893177,71576,0.0,"POLYGON ((128.51925 38.15510, 128.52039 38.155...","[[128.51925287359862, 38.155103313580724], [12...","[128.51982975576533, 38.155548920969245]",POINT (128.5198297557653 38.15554892096925),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
71577,라아857163,71577,0.0,"POLYGON ((128.47800 38.14284, 128.47914 38.142...","[[128.4780003347657, 38.14283679526627], [128....","[128.47857687210077, 38.14328261097056]",POINT (128.4785768721008 38.14328261097056),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
71578,라아890218,71578,0.0,"POLYGON ((128.51634 38.19208, 128.51748 38.192...","[[128.51634225100835, 38.1920789478286], [128....","[128.51691941246253, 38.19252456289475]",POINT (128.5169194124625 38.19252456289475),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
71579,라아854182,71579,0.0,"POLYGON ((128.47481 38.15999, 128.47595 38.159...","[[128.4748050967108, 38.15998683245551], [128....","[128.47538175223931, 38.16043266099766]",POINT (128.4753817522393 38.16043266099766),0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### 분석 결과
- 금속충전소와 완속충전소는 고려되는 요소들의 영향이 차이가 있다.
- 급속충전소 경우, 정규화_혼잡빈도강도합에 가장 많은 영향을 받았고, 완속충전소 경우, 정규화_인구에 가장 많은 영향을 받았다. 
- 이 결과는 차량이 많이 다는 곳에 급속 충전소를 설치하고, 주거 공간이 많이 있는 곳에 완속 충전소를 설치했다고 해석할 수 있다.
- 완속 충전소의 경우 급속충전소가 가장 크게 영향을 받은 정규화_혼잡빈도강도합의 변수값이 음수로 분석되었으며, 이는 혼잡한 곳에는 오히려 완속충전소가 설치 되지 않는 것을 의미한다. 
- 위 분석 결과는 상식적인 부분과 잘 맞는다.
- 따라서 위 결과를 이용하여 추후 최적화 모델의 목적함수 변수 값으로 사용하기 위해 w_fs, w_ss로 가중치값을 계산하였다. 
-------------------------------------



# III. 최적화 문제 정의 및 해결

## 전기자동차 충전소 위치 선정 최적화 모델 정의 및 결과

### 최적화 모델 : Maximal Covering Location Problem (MCLP)

- 정의: MCLP는 최대지역커버문제로, 설비가 커버하는 수요 (covered demand)의 합을 최대화 하면서 주어진 K개의 설비를 세울 위치를 선정하는 문제 
- 가정

    - 설비의 위치가 수요 발생 지점으로부터 일정 거리 Residual 이내에 수요를 커버함. 
    - 이때 거리 Residual은 커버리지 거리(covered distance) 라고 함.
    - 커버되지 못한 수여는 서비스를 받지 못하는 수요가 아니라 서비스를 받긴 하지만 서비스 받는 설비로 부터의 거리가 커버리지 밖에 있어 만족할 만한 서비스 수준을 제공받지 못하는 수요를 의미
    
    
- Mathematical statement


    - i : 수요 포인트 index
    - j : 설비 후보지역 index
    - I : 수요 포인트 집합
    - J : 설비 후보지역 집합
    - K : 총 설치해야 하는 설비 개수
    - x : 설비 후보 지역 중 위치 j에 설비가 설치되면 1, 그렇지 않으면 0
    - y : 적어도 하나의 설비로 그 포인트가 커버가 되면 1, 그렇지 않으면 0
    
    
- Formulation

$$
\begin{align*}
&\text{maximize} \sum_{i\in I} w_i y_i ...(1) \\
\text{s.t.} \quad & y_i \le \sum_{j\in N_i}x_j \qquad for \quad all \quad i\in I ... (2)\\
&\sum_{j\in J}x_j = K ... (3)\\
&x_j, y_i \in \{0,1\} \qquad for \quad all \quad i\in I,j\in J 
\end{align*}
$$
    

    -(1) : 목적함수, 가중치 w인 수요 포인트를 최대한 많이 커버하게 해라
    -(2) : 수요포인트 i는 설비 후보 지역이 커버하는 거리안에서 적어도 하나 이상의 설비로 부터 커버가 된다. 
    -(3) : 총 설치할 설비는 K개 이다.


In [57]:
def generate_candidate_sites(points,M=100):
    '''
    Generate M candidate sites with the convex hull of a point set
    Input:
        points: a Numpy array with shape of (N,2)
        M: the number of candidate sites to generate
    Return:
        sites: a Numpy array with shape of (M,2)
    '''
    hull = ConvexHull(points)
    polygon_points = points[hull.vertices]
    poly = Polygon(polygon_points)
    min_x, min_y, max_x, max_y = poly.bounds
    sites = []
    while len(sites) < M:
        random_point = Point([random.uniform(min_x, max_x),
                             random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])




def generate_candidate_sites(df_result_fin,M=100):
    from shapely.geometry import Polygon, Point
    sites = []
    idx=np.random.choice(np.array(range(0,len(df_result_fin))), M)
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.iloc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.iloc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])

def generate_candidate_sites(df_result_fin,Weight,M=100):
    sites = []
    idx = df_result_fin.sort_values(by = Weight, ascending = False).iloc[1:M].index
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.loc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.loc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])



from scipy.spatial import distance_matrix
def mclp(points,K,radius,M,df_result_fin,w,Weight):

    """
    Solve maximum covering location problem
    Input:
        points: input points, Numpy array in shape of [N,2]
        K: the number of sites to select
        radius: the radius of circle
        M: the number of candidate sites, which will randomly generated inside
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    print('----- Configurations -----')
    print('  Number of points %g' % points.shape[0])
    print('  K %g' % K)
    print('  Radius %g' % radius)
    print('  M %g' % M)
    import time
    start = time.time()
    sites = generate_candidate_sites(df_result_fin,Weight,M)
    J = sites.shape[0]
    I = points.shape[0]
    D = distance_matrix(points,sites)
    mask1 = D<=radius
    D[mask1]=1
    D[~mask1]=0

    from mip import Model, xsum, maximize, BINARY

    # Build model
    m = Model("mclp")
    # Add variables

    x = [m.add_var(name = "x%d" % j, var_type = BINARY) for j in range(J)]
    y = [m.add_var(name = "y%d" % i, var_type = BINARY) for i in range(I)]


    m.objective = maximize(xsum(w[i]*y[i] for i in range (I)))

    m += xsum(x[j] for j in range(J)) == K

    for i in range(I):
        m += xsum(x[j] for j in np.where(D[i]==1)[0]) >= y[i]

    m.optimize()
    
    end = time.time()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objective_value)

    solution = []
    for i in range(J):
        if x[i].x ==1:
            solution.append(int(x[i].name[1:]))
    opt_sites = sites[solution]
            
    return opt_sites,m.objective_value



In [58]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#import xgboost
import sklearn.cluster
import tensorflow as tf

#from geoband import API

import pydeck as pdk
import os

import pandas as pd


import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
#import deckgljupyter.Layer as deckgl

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'

import numpy as np
from shapely.geometry import Polygon, Point
from numpy import random

#최적화 solver
import time
from mip import Model, xsum, maximize, BINARY  

df_test=gpd.read_file('df_result.geojson')
df_test
# 100X100 grid에서 central point 찾기
df_list = []
df_list2 = []
for i in df_test['geometry']:
    cent = [[i.centroid.coords[0][0],i.centroid.coords[0][1]]]
    df_list.append(cent)
    df_list2.append(Point(cent[0]))
df_test['coord_cent'] = 0
df_test['geo_cent'] = 0
df_test['coord_cent']= pd.DataFrame(df_list) # pydeck을 위한 coordinate type
df_test['geo_cent'] = df_list2 # geopandas를 위한 geometry type
df_test

,grid_id,정규화_교통량_11,정규화_교통량_14,정규화_혼잡빈도강도합,정규화_혼잡시간강도합,정규화_관광지_score_stay,정규화_관광지_네비검색량,정규화_관광지_방문객수,정규화_관광지_동_유동인구,정규화_상권_score_search_point,정규화_상권_score_view_count,정규화_상권_score_review_count,정규화_상권_score_stay,정규화_상권_동_유동인구,w_FS,w_SS,개발가능,FS_station,SS_station,geometry,coord_cent,geo_cent
0,00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103093,0.096898,0.100423,0.180617,0.023149,0.051670,0.087201,1,0,0,"POLYGON ((128.45110 38.44316, 128.45225 38.443...","[128.45167991472766, 38.44360845418272]",POINT (128.4516799147277 38.44360845418272)
1,00001,0.000000,0.000000,0.001497,0.004388,0.000000,0.000000,0.000000,0.000000,0.010309,0.010016,0.010365,0.088106,0.009262,0.006710,0.024401,0,0,0,"POLYGON ((128.53675 38.26220, 128.53789 38.262...","[128.53732774651786, 38.262641181764714]",POINT (128.5373277465179 38.26264118176471)
2,00002,0.000000,0.000000,0.021856,0.036753,0.341691,0.010331,0.040114,0.006974,0.010309,0.010016,0.010365,0.010279,0.002580,0.491978,0.267937,0,0,0,"POLYGON ((128.42322 38.50286, 128.42436 38.502...","[128.42379424556938, 38.50331020336737]",POINT (128.4237942455694 38.50331020336737)
3,00003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,"POLYGON ((128.54981 38.29633, 128.55096 38.296...","[128.55039276952658, 38.296770983303354]",POINT (128.5503927695266 38.29677098330335)
4,00004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,"POLYGON ((128.42280 38.47042, 128.42395 38.470...","[128.4233801674755, 38.47087068851492]",POINT (128.4233801674755 38.47087068851492)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71576,71576,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,"POLYGON ((128.51925 38.15510, 128.52039 38.155...","[128.51982975576533, 38.155548920969245]",POINT (128.5198297557653 38.15554892096925)
71577,71577,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,"POLYGON ((128.47800 38.14284, 128.47914 38.142...","[128.47857687210077, 38.14328261097056]",POINT (128.4785768721008 38.14328261097056)
71578,71578,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,"POLYGON ((128.51634 38.19208, 128.51748 38.192...","[128.51691941246253, 38.19252456289475]",POINT (128.5169194124625 38.19252456289475)
71579,71579,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,"POLYGON ((128.47481 38.15999, 128.47595 38.159...","[128.47538175223931, 38.16043266099766]",POINT (128.4753817522393 38.16043266099766)


In [59]:
%%time
## 급속충전소 500m반경
df_result_fin = df_test[(df_test['개발가능']==1)
                          &(df_test['FS_station']!=1)]
df_result_fin

points = []
for i in df_result_fin['coord_cent'] :
    points.append(i)

w= []
for i in df_result_fin['w_FS'] :
    w.append(i)

radius = radius = (1/88.74/1000)*500     ## 500m 반경을 표현함
K = 50  ## 총 설치해야하는 설비 개수
M = 5000  ## 급속 충전소 입지선정지수가 가장 높은 5000개 point

opt_sites_org,f = mclp(np.array(points),K,radius,M,df_result_fin,w,'w_FS')


df_opt_FS= pd.DataFrame(opt_sites_org)
df_opt_FS.columns = ['lon', 'lat']
df_opt_FS

----- Configurations -----
  Number of points 8756
  K 50
  Radius 0.00563444
  M 5000
----- Output -----
  Running time : 44.1321120262146 seconds
  Optimal coverage points: 357.398
CPU times: user 37.1 s, sys: 3.08 s, total: 40.1 s
Wall time: 44.2 s


In [60]:
layer0 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
                  df_14_possible, # 시각화에 쓰일 데이터프레임
                  #df_result_fin[df_result_fin['val']!=0],
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[0, 255*1, 0,140]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 


# layer1 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
#                   df_14_possible, # 시각화에 쓰일 데이터프레임
#                   #df_result_fin[df_result_fin['val']!=0],
#                   get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
#                   get_fill_color='[0, 255*1, 0,140]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 
#                   pickable=True, # 지도와 interactive 한 동작 on 
#                   auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
#                  ) 

# layer2 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
#                   df_08[(df_08['val'].isnull()==False) & df_08['val']!=0], # 시각화에 쓰일 데이터프레임 
#                   get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
#                   get_fill_color='[900, 255*정규화인구, 0, 정규화인구*10000 ]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#                   pickable=True, # 지도와 interactive 한 동작 on 
#                   auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
#                  )  

# layer3 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
#                   df_14_possible, # 시각화에 쓰일 데이터프레임
#                   #df_result_fin[df_result_fin['val']!=0],
#                   get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
#                   get_fill_color='[0, 255*1, 0,140]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 
#                   pickable=True, # 지도와 interactive 한 동작 on 
#                   auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
#                  ) 

# layer4 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
#                   df_08[(df_08['val'].isnull()==False) & df_08['val']!=0], # 시각화에 쓰일 데이터프레임 
#                   get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
#                   get_fill_color='[900, 255*정규화인구, 0, 정규화인구*10000 ]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#                   pickable=True, # 지도와 interactive 한 동작 on 
#                   auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
#                  )  

# layer5 = pdk.Layer( 'PathLayer', 
#                   df_10_11_time14, 
#                   get_path='coordinate', 
#                   get_width='교통량/2', 
#                   get_color='[255, 255 * 정규화도로폭, 120]', 
#                   pickable=True, auto_highlight=True 
#                  ) 


# layer6 = pdk.Layer( 'PathLayer', 
#                   df_10_12, 
#                   get_path='coordinate', 
#                   get_width='혼잡빈도강도합/2', 
#                   get_color='[255, 255 * 정규화도로폭, 120,140]', 
#                   pickable=True, auto_highlight=True 
#                  ) 


# layer7 = pdk.Layer( 'PathLayer', 
#                   df_10_13, 
#                   get_path='coordinate', 
#                   get_width='혼잡시간강도합/10', 
#                   get_color='[255, 255 * 정규화도로폭, 120]', 
#                   pickable=True, auto_highlight=True 
#                  ) 

# Set the viewport location 
center = [128.5918, 38.27701] # 속초 센터 [128.5918, 38.20701]
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 

## 기존 충전소 위치
scatt = pdk.Layer(
    'ScatterplotLayer',
    df_01_geo[df_01_geo['급속/완속']=='급속'][['lon','lat']],
    get_position = ['lon','lat'],
    auto_highlight=True,
    get_radius=100,
    get_fill_color='[50, 50, 200]',
    pickable=True)
## 최적화 충전소 위치
opt = pdk.Layer(
    'ScatterplotLayer',
    df_opt_FS,
    get_position = ['lon','lat'],
    auto_highlight=True,
    get_radius=100,
    get_fill_color='[255, 255, 0]',
    get_line_color = '[0, 0, 0]',
    line_width_min_pixels=5,
    pickable=True)



# Render 
r = pdk.Deck(layers=[layer0,scatt,opt], initial_view_state=view_state)
#             mapbox_key = "pk.eyJ1IjoiamNsYXJhODExIiwiYSI6ImNrZzF4bWNhdTBpNnEydG54dGpxNDEwajAifQ.XWxOKQ-2HqFBVBYa-XoS-g"

    
r.to_html('급속_반경500m.html')


## 노란색 : 제안된 최적화 지역
## 파란색 : 기존 급속 충전소

Output hidden; open in https://colab.research.google.com to view.

In [61]:
###### 완속 충전소




# df_result_fin = df_test[(df_test['SS_station']!=1)]
# df_result_fin

# points = []
# for i in df_result_fin['coord_cent'] :
#     points.append(i)

# w= []
# for i in df_result_fin['w_SS'] :
#     w.append(i)

# radius = (1/88.74/1000)*50   
# K = 30
# M = 5000

# opt_sites_org,f = mclp(np.array(points),K,radius,M,df_result_fin,w,'w_SS')


# df_opt_SS= pd.DataFrame(opt_sites_org)
# df_opt_SS.columns = ['lon', 'lat']
# df_opt_SS



# # Make layer 

# layer = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
#                   df_14_possible, # 시각화에 쓰일 데이터프레임
#                   #df_result_fin[df_result_fin['val']!=0],
#                   get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
#                   get_fill_color='[0, 255*1, 0,140]', # 각 데이터 별 rgb 또는 rgba 값 (0~255) 
#                   pickable=True, # 지도와 interactive 한 동작 on 
#                   auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
#                  ) 

# layer0 = pdk.Layer( 'PolygonLayer', # 사용할 Layer 타입 
#                   df_08[(df_08['val'].isnull()==False) & df_08['val']!=0], # 시각화에 쓰일 데이터프레임 
#                   get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
#                   get_fill_color='[900, 255*정규화인구, 0, 정규화인구*10000 ]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
#                   pickable=True, # 지도와 interactive 한 동작 on 
#                   auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
#                  )  

# # layer1 = pdk.Layer( 'PathLayer', 
# #                   df_10_11_time14, 
# #                   get_path='coordinate', 
# #                   get_width='교통량/2', 
# #                   get_color='[255, 255 * 정규화도로폭, 120]', 
# #                   pickable=True, auto_highlight=True 
# #                  ) 


# # layer2 = pdk.Layer( 'PathLayer', 
# #                   df_10_12, 
# #                   get_path='coordinate', 
# #                   get_width='혼잡빈도강도합/2', 
# #                   get_color='[255, 255 * 정규화도로폭, 120,140]', 
# #                   pickable=True, auto_highlight=True 
# #                  ) 


# layer3 = pdk.Layer( 'PathLayer', 
#                   df_10_13, 
#                   get_path='coordinate', 
#                   get_width='혼잡시간강도합/2', 
#                   get_color='[255, 255 * 정규화도로폭, 120]', 
#                   pickable=True, auto_highlight=True 
#                  ) 


# # Set the viewport location 
# center = [128.5918, 38.27701] # 속초 센터 [128.5918, 38.20701]
# view_state = pdk.ViewState( 
#     longitude=center[0], 
#     latitude=center[1], 
#     zoom=10
# ) 


# scatt = pdk.Layer(
#     'ScatterplotLayer',
#     df_01_geo[df_01_geo['급속/완속']=='완속'][['lon','lat']],
#     get_position = ['lon','lat'],
#     auto_highlight=True,
#     get_radius=200,
#     get_fill_color='[100, 200, 100,140]',
#     pickable=True)

# opt = pdk.Layer(
#     'ScatterplotLayer',
#     df_opt_SS,
#     get_position = ['lon','lat'],
#     auto_highlight=True,
#     get_radius=200,
#     get_fill_color='[255, 255, 0]',
#     get_line_color = '[0, 0, 0]',
#     line_width_min_pixels=5,
#     pickable=True)



# # Render 
# r = pdk.Deck(layers=[layer,layer0,layer3,scatt,opt], initial_view_state=view_state)
# #             mapbox_key = "pk.eyJ1IjoiamNsYXJhODExIiwiYSI6ImNrZzF4bWNhdTBpNnEydG54dGpxNDEwajAifQ.XWxOKQ-2HqFBVBYa-XoS-g") 


    
# r.to_html('녹색_기존완속충전소_노란색_제안된 완속충전소 최적화지역.html')

# ## 노란색 : 제안된 최적화 지역
# ## 녹색 : 기존 완속 충전소

In [62]:
df_opt_FS['충전소구분']='급속'
# df_opt_SS['충전소구분']='완속'
pd.concat([df_opt_FS]).to_csv("충전소결과.csv", index=False)